In [1]:
from bs4 import BeautifulSoup
from bs4.formatter import HTMLFormatter
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math
import pandas as pd

In [2]:
state_to_abbreviation = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [3]:
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
# get the state names
election_result = requests.get("https://www.politico.com/2022-election/results/", timeout=3)
bs_election = BeautifulSoup(election_result.content, 'html.parser')
driver.get("https://www.politico.com/2022-election/results/")
wait = WebDriverWait(driver, 10)

# state_list
states = bs_election.find_all('div', {'class': 'styles_column__Twooh styles_is-mobile__c1oGv'})
state_name_list = []
for state in states:
    text = state.a.get_text(strip=True)
    for i in state:
        state_name_list.append(i.get_text(strip=True))
print(state_name_list)

[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.81M/6.81M [00:15<00:00, 475kB/s]


['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [4]:
# state_link
from selenium.common.exceptions import TimeoutException
state_list = [state.replace(' ', '-') for state in state_name_list]

state_link=[]
for state in state_list:
    state_link.append("https://www.politico.com/2022-election/results/"+state.lower()+"/house")
state_link

['https://www.politico.com/2022-election/results/alabama/house',
 'https://www.politico.com/2022-election/results/alaska/house',
 'https://www.politico.com/2022-election/results/arizona/house',
 'https://www.politico.com/2022-election/results/arkansas/house',
 'https://www.politico.com/2022-election/results/california/house',
 'https://www.politico.com/2022-election/results/colorado/house',
 'https://www.politico.com/2022-election/results/connecticut/house',
 'https://www.politico.com/2022-election/results/delaware/house',
 'https://www.politico.com/2022-election/results/florida/house',
 'https://www.politico.com/2022-election/results/georgia/house',
 'https://www.politico.com/2022-election/results/hawaii/house',
 'https://www.politico.com/2022-election/results/idaho/house',
 'https://www.politico.com/2022-election/results/illinois/house',
 'https://www.politico.com/2022-election/results/indiana/house',
 'https://www.politico.com/2022-election/results/iowa/house',
 'https://www.politic

In [20]:
# for loop

state_table_list = pd.DataFrame(columns=['State', 'State Code', 'District', 'Party', 'Candidate', 'Incumbent', 'Vote', 'Pct%'])


for k in range(len(state_link)):
    driver = webdriver.Chrome()
    driver.get(state_link[k])
    wait = WebDriverWait(driver, 10)  
    buttons = driver.find_elements(By.CSS_SELECTOR, 'button.styles_container__C5E_O[title="Expand"]')
    while len(buttons) > 0:
        for button in buttons:
            button.click()
        buttons = driver.find_elements(By.CSS_SELECTOR, 'button.styles_container__C5E_O[title="Expand"]')


    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')


    # Extract the candidate names
    scraped_data = []
    data = soup.find_all('div', {'class':'styles_table-container__vTHda'})
    spans = soup.find_all('span')

    candidate_list = []
    candidate_list_for_district = []
    party_list = []
    incumbent_list = []
    vote_list = []
    pct_list = []
    state_code_list = []
    state_list = []
    candidate_list_for_district = []

    # text_list for candidate_list and more
    text_list = []
    for result in data:
        spans = result.find_all('span', class_=False)
        for span in spans:
            text_list.append(span.text)

    # make text_list prettier
    text_list = [text for text in text_list if not text[0].isdigit()]
    text_list = [name.replace("(R)", "(Republican)").replace("(D)", "(Democratic)") for name in text_list]
    text_list = [text for text in text_list if 'Hide other candidates' not in text]
    
    # candidate_list - done 
    for i in text_list:
        candidate = i.split(" (")[0].rstrip("*")
        candidate_list.append(candidate)

    # party_list - done
    for i in text_list:
        start = i.find('(')
        end = i.find(')')
        if start != -1 and end != -1:
            party = i[start+1:end]
        else:
            party = "/"
        party_list.append(party)

    # incumbent_list - done
    for i in text_list:
        if "*" in i:
            incumbent_list.append("Incumbent")
        else:
            incumbent_list.append("/")
    # vote_list
    for result in data:
        votes = result.find_all("td", class_=["styles_container__MY5SI", "styles_container__7QaMH"])
        for vote in votes:
            if vote.text == "Uncontested":
                vote_list.append('/')
            else:
                vote_list.append(vote.text)

    # pct_list - done
    for result in data:
        percentage = result.find_all("td", class_=["styles_container__vzwvV", "styles_container__7QaMH"])
        for pct in percentage:
            if pct.text == "Uncontested":
                pct_list.append('/')
            else:
                pct_list.append(pct.text)

    for i in range(len(party_list)):
        state_code_list.append(state_to_abbreviation[state_name_list[k]])
        state_list.append(state_name_list[k])
        

    # district_list
    for j in candidate_list:
        candidate_list_for_district.append(j.split(" (")[0].replace("*",""))      

    dem_win=soup.find_all("span",{"class":"styles_highlighted__YhtNx styles_contrast-dem__WYCAC"})
    dem_win_ls=[]
    for i in dem_win:
        dem_win_ls.append(i.text)

    gop_win_ls=[]
    gop_win=soup.find_all("span",{"class":["styles_highlighted__YhtNx styles_contrast-dem__WYCAC", "styles_highlighted__YhtNx styles_contrast-gop__2dV83"]})
    for i in gop_win:
        gop_win_ls.append(i.text)

    win_ls=gop_win_ls+dem_win_ls
    win_name_ls=[]
    for i in win_ls:
        win_name_ls.append(i.split(" (")[0].replace("*",""))
    
    district_num = []
    districts = soup.find_all("h5", {"class": "styles_is-table__D_lzx styles_is-desktop__4Cq7X"})
    if not districts:
        district_num.append("/")
    else:
        for district in districts:
            district_num.append(district.get_text())

     
    district_list = []
    h = [i in win_name_ls for i in candidate_list_for_district]
    repeated_district_ls = []

    try:
        repeated_district_ls = [district_num[sum(h[:i+1])-1] for i in range(len(candidate_list_for_district))]
    except IndexError:
        # Handle the IndexError by appending the district number to district_list
        for i in range(len(candidate_list_for_district)):
            try:
                district = district_num[sum(h[:i+1])-1]
                district_list.append(district)
            except IndexError:
                district_list.append(district)


    district_list += repeated_district_ls

    
    list_lengths = [len(district_list), len(party_list), len(candidate_list), len(incumbent_list), len(vote_list), len(pct_list)]

    min_length = min(list_lengths)

    if len(district_list) > min_length:
        district_list.pop()
    if len(state_list) > min_length:
        state_list.pop()
    if len(state_code_list) > min_length:
        state_code_list.pop()
    if len(party_list) > min_length:
        party_list.pop()
    if len(candidate_list) > min_length:
        candidate_list.pop()
    if len(incumbent_list) > min_length:
        incumbent_list.pop()
    if len(vote_list) > min_length:
        vote_list.pop(0)
    if len(pct_list) > min_length:
        pct_list.pop(0)

    table = pd.DataFrame({
        'State': state_list[:min_length],
        'State Code': state_code_list[:min_length],
        'District': district_list[:min_length],
        'Party': party_list,
        'Candidate': candidate_list,
        'Incumbent': incumbent_list,
        'Vote': vote_list,
        'Pct%': pct_list
    })
    
    print(table)
    state_table_list = pd.concat([state_table_list, table])
    
    


      State State Code District              Party       Candidate  Incumbent  \
0   Alabama         AL      1st         Republican            Carl  Incumbent   
1   Alabama         AL      1st  Libertarian Party          Remrey          /   
2   Alabama         AL      2nd         Republican           Moore  Incumbent   
3   Alabama         AL      2nd         Democratic     Harvey-Hall          /   
4   Alabama         AL      2nd  Libertarian Party           Realz          /   
5   Alabama         AL      3rd         Republican          Rogers  Incumbent   
6   Alabama         AL      3rd         Democratic          Veasey          /   
7   Alabama         AL      3rd                  /           Bell           /   
8   Alabama         AL      3rd  Libertarian Party          Casson          /   
9   Alabama         AL      4th         Republican        Aderholt  Incumbent   
10  Alabama         AL      4th         Democratic       Neighbors          /   
11  Alabama         AL      

          State State Code District              Party  Candidate  Incumbent  \
0   Connecticut         CT      1st         Democratic     Larson  Incumbent   
1   Connecticut         CT      1st         Republican      Lazor          /   
2   Connecticut         CT      1st        Green Party    Sanders          /   
3   Connecticut         CT      2nd         Democratic   Courtney  Incumbent   
4   Connecticut         CT      2nd         Republican     France          /   
5   Connecticut         CT      2nd        Green Party    Blacker          /   
6   Connecticut         CT      2nd  Libertarian Party       Hall          /   
7   Connecticut         CT      3rd         Democratic    DeLauro  Incumbent   
8   Connecticut         CT      3rd         Republican   DeNardis          /   
9   Connecticut         CT      3rd               Ind.       Chai          /   
10  Connecticut         CT      3rd        Green Party    Paglino          /   
11  Connecticut         CT      4th     

      State State Code District              Party  Candidate  Incumbent  \
0   Indiana         IN      1st         Democratic      Mrvan  Incumbent   
1   Indiana         IN      1st         Republican      Green          /   
2   Indiana         IN      2nd         Republican      Yakym          /   
3   Indiana         IN      2nd         Democratic     Steury          /   
4   Indiana         IN      2nd  Libertarian Party      Henry          /   
5   Indiana         IN      2nd         Republican      Yakym          /   
6   Indiana         IN      2nd         Democratic     Steury          /   
7   Indiana         IN      2nd  Libertarian Party      Henry          /   
8   Indiana         IN      3rd         Republican      Banks  Incumbent   
9   Indiana         IN      3rd         Democratic     Snyder          /   
10  Indiana         IN      3rd               Ind.     Gotsch          /   
11  Indiana         IN      4th         Republican      Baird  Incumbent   
12  Indiana 

       State State Code District              Party      Candidate  Incumbent  \
0   Maryland         MD      1st         Republican         Harris  Incumbent   
1   Maryland         MD      1st         Democratic         Mizeur          /   
2   Maryland         MD      1st  Libertarian Party      Thibeault          /   
3   Maryland         MD      2nd         Democratic  Ruppersberger  Incumbent   
4   Maryland         MD      2nd         Republican        Ambrose          /   
5   Maryland         MD      3rd         Democratic       Sarbanes  Incumbent   
6   Maryland         MD      3rd         Republican         Morgan          /   
7   Maryland         MD      4th         Democratic           Ivey          /   
8   Maryland         MD      4th         Republican         Warner          /   
9   Maryland         MD      5th         Democratic          Hoyer  Incumbent   
10  Maryland         MD      5th         Republican        Palombi          /   
11  Maryland         MD     

        State State Code District       Party    Candidate  Incumbent  \
0   Minnesota         MN      1st  Republican      Finstad  Incumbent   
1   Minnesota         MN      1st  Democratic     Ettinger          /   
2   Minnesota         MN      1st           /    Reisdorf           /   
3   Minnesota         MN      1st           /  Abrahamson           /   
4   Minnesota         MN      2nd  Democratic        Craig  Incumbent   
5   Minnesota         MN      2nd  Republican      Kistner          /   
6   Minnesota         MN      2nd           /      Overby           /   
7   Minnesota         MN      3rd  Democratic     Phillips  Incumbent   
8   Minnesota         MN      3rd  Republican       Weiler          /   
9   Minnesota         MN      4th  Democratic     McCollum  Incumbent   
10  Minnesota         MN      4th  Republican        Xiong          /   
11  Minnesota         MN      5th  Democratic         Omar  Incumbent   
12  Minnesota         MN      5th  Republican      

         State State Code District       Party       Candidate  Incumbent  \
0   New Jersey         NJ      1st  Democratic        Norcross  Incumbent   
1   New Jersey         NJ      1st  Republican       Gustafson          /   
2   New Jersey         NJ      1st        Ind.           Kline          /   
3   New Jersey         NJ      1st        Ind.        Fletcher          /   
4   New Jersey         NJ      1st        Ind.          Cannon          /   
5   New Jersey         NJ      2nd  Republican        Van Drew  Incumbent   
6   New Jersey         NJ      2nd  Democratic       Alexander          /   
7   New Jersey         NJ      2nd        Ind.           Gallo          /   
8   New Jersey         NJ      2nd        Ind.         Sanchez          /   
9   New Jersey         NJ      3rd  Democratic             Kim  Incumbent   
10  New Jersey         NJ      3rd  Republican          Healey          /   
11  New Jersey         NJ      3rd        Ind.      Russomanno          /   

             State State Code District              Party      Candidate  \
0   North Carolina         NC      1st         Democratic          Davis   
1   North Carolina         NC      1st         Republican          Smith   
2   North Carolina         NC      2nd         Democratic           Ross   
3   North Carolina         NC      2nd         Republican     Villaverde   
4   North Carolina         NC      3rd         Republican         Murphy   
5   North Carolina         NC      3rd         Democratic        Gaskins   
6   North Carolina         NC      4th         Democratic        Foushee   
7   North Carolina         NC      4th         Republican          Geels   
8   North Carolina         NC      5th         Republican           Foxx   
9   North Carolina         NC      5th         Democratic        Parrish   
10  North Carolina         NC      6th         Democratic        Manning   
11  North Carolina         NC      6th         Republican       Castelli   
12  North Ca

           State State Code District       Party      Candidate  Incumbent  \
0   Pennsylvania         PA      1st  Republican    Fitzpatrick  Incumbent   
1   Pennsylvania         PA      1st  Democratic          Ehasz          /   
2   Pennsylvania         PA      2nd  Democratic          Boyle  Incumbent   
3   Pennsylvania         PA      2nd  Republican         Bashir          /   
4   Pennsylvania         PA      3rd  Democratic          Evans  Incumbent   
5   Pennsylvania         PA      3rd           /      Hoeppner           /   
6   Pennsylvania         PA      4th  Democratic           Dean  Incumbent   
7   Pennsylvania         PA      4th  Republican     Nascimento          /   
8   Pennsylvania         PA      5th  Democratic        Scanlon  Incumbent   
9   Pennsylvania         PA      5th  Republican        Galluch          /   
10  Pennsylvania         PA      6th  Democratic       Houlahan  Incumbent   
11  Pennsylvania         PA      6th  Republican     Ciarrocchi 

    State State Code District              Party      Candidate  Incumbent  \
0   Texas         TX      1st         Republican          Moran          /   
1   Texas         TX      1st         Democratic      Jefferson          /   
2   Texas         TX      2nd         Republican       Crenshaw  Incumbent   
3   Texas         TX      2nd         Democratic        Fulford          /   
4   Texas         TX      3rd         Republican           Self          /   
..    ...        ...      ...                ...            ...        ...   
83  Texas         TX     38th         Republican  Garcia Sharon          /   
84  Texas         TX     38th  Libertarian Party      Patterson          /   
85  Texas         TX     38th         Republican           Hunt          /   
86  Texas         TX     38th         Democratic      Klussmann          /   
87  Texas         TX     38th               Ind.         Dejean          /   

       Vote   Pct%  
0   183,005  78.1%  
1    51,288  21.9%  


        State State Code District              Party    Candidate  Incumbent  \
0   Wisconsin         WI      1st         Republican        Steil  Incumbent   
1   Wisconsin         WI      1st         Democratic          Roe          /   
2   Wisconsin         WI      1st               Ind.       Barman          /   
3   Wisconsin         WI      2nd         Democratic        Pocan  Incumbent   
4   Wisconsin         WI      2nd         Republican        Olsen          /   
5   Wisconsin         WI      2nd               Ind.    Alexander          /   
6   Wisconsin         WI      3rd         Republican    Van Orden          /   
7   Wisconsin         WI      3rd         Democratic        Pfaff          /   
8   Wisconsin         WI      4th         Democratic        Moore  Incumbent   
9   Wisconsin         WI      4th         Republican       Rogers          /   
10  Wisconsin         WI      4th               Ind.      Raymond          /   
11  Wisconsin         WI      5th       

In [21]:
state_table_list

,State,State Code,District,Party,Candidate,Incumbent,Vote,Pct%
0,Alabama,AL,1st,Republican,Carl,Incumbent,"139,854",84.2%
1,Alabama,AL,1st,Libertarian Party,Remrey,/,"26,197",15.8%
2,Alabama,AL,2nd,Republican,Moore,Incumbent,"137,193",69.2%
3,Alabama,AL,2nd,Democratic,Harvey-Hall,/,"57,800",29.1%
4,Alabama,AL,2nd,Libertarian Party,Realz,/,"3,380",1.7%
...,...,...,...,...,...,...,...,...
18,Wisconsin,WI,8th,Libertarian Party,VandenPlas,/,"32,043",10.5%
0,Wyoming,WY,/,Republican,Harriet Hageman,/,"132,215",69.8%
1,Wyoming,WY,/,Democratic,Lynnette GreyBull,/,"47,241",24.9%
2,Wyoming,WY,/,Libertarian Party,Richard Brubaker,/,"5,420",2.9%


In [22]:
state_table_list.to_csv('house.csv', index=False)